In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re

In [2]:
data = pd.read_excel('sentence_past_present_future.xlsx')

In [3]:
data

,Sentences,Categorie
0,Lisa went to the supermarket yesterday.,Past
1,Sam cooked a tasty dinner yesterday.,Past
2,My brother saw a movie yesterday.,Past
3,"Last year, I travelled to France.",Past
4,I washed the dishes.,Past
...,...,...
2764,He will have been complaining to the plumber a...,Future
2765,The poet will have been writing a romantic poe...,Future
2766,Will she have been eating dinner when I enter ...,Future
2767,The lyricist will have been writing a realisti...,Future


In [4]:
y = data['Categorie']

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)


In [6]:
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)

In [7]:
input

[[5657, 1324, 1563, 8161, 5010, 8729, 5552],
 [7102, 6751, 1424, 3129, 9571, 8729, 5552],
 [1248, 8984, 4794, 1424, 3250, 8729, 5552],
 [9603, 8621, 5552, 633, 936, 1563, 8772, 5552],
 [633, 3016, 8161, 9321, 5552],
 [1248, 5548, 225, 1424, 1695, 4875, 6186, 5552],
 [7102, 3332, 6186, 7999, 5582, 4875, 8895, 3264, 5552],
 [633, 265, 683, 1563, 2270, 8161, 6987, 5146, 5231, 7921, 5552],
 [1248, 8984, 5237, 6414, 8866, 8161, 8505, 1796, 5552],
 [7066, 1000, 1563, 7751, 7289, 4875, 8895, 6414, 5340, 8161, 5676, 5552],
 [1248, 5654, 2252, 1563, 301, 471, 2530, 8621, 5552],
 [633, 2849, 8847, 1343, 633, 6569, 8127, 5552],
 [1349, 7895, 6125, 1424, 7007, 4413, 506, 6569, 6186, 1133, 5552],
 [7098, 3431, 1220, 1563, 4076, 1343, 3431, 8950, 1424, 6972, 5552],
 [7380, 4123, 6809, 1343, 8340, 6569, 6170, 3264, 8787, 5552],
 [1248, 7289, 49, 1563, 9929, 1343, 506, 6569, 6186, 7598, 5552],
 [1248, 6842, 2458, 1882, 1343, 506, 6569, 1424, 7084, 5552],
 [286, 8144, 5773, 1608, 1971, 2889, 6125, 1424

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 5010, 8729, 5552],
       [   0,    0,    0, ..., 9571, 8729, 5552],
       [   0,    0,    0, ..., 3250, 8729, 5552],
       ...,
       [   0,    0,    0, ..., 8161, 7758, 5552],
       [   0,    0, 5087, ..., 8161, 4931, 5552],
       [   0,    0,    0, ..., 7751, 5113, 5552]])

In [9]:
dim = 10

In [10]:
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
x = model.predict(embedded_docs)

87/87 [==============================] - 0s 1ms/step


In [12]:
rows = x.shape[0]
cols = x.shape[1] * x.shape[2]
x = x.reshape(rows,cols)

In [13]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
knn = KNeighborsClassifier(metric='euclidean')

In [14]:
knn.fit(x_train,y_train)

KNeighborsClassifier(metric='euclidean')

In [15]:
knn.score(x_test,y_test)

0.6551226551226551

In [16]:
y_pred = knn.predict(x_test)

In [17]:
# accuracy , precision , recall , f1-score , confusion matrix , roc auc

In [18]:
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , roc_auc_score

In [51]:
def Evaluation_metrics(actual,pred):
    accuracy_sco = accuracy_score(actual,pred)
    confusion_mat = confusion_matrix(actual,pred)
    precision_sco = precision_score(actual,pred,average='macro')
    recall_sco = recall_score(actual,pred,average='macro')
    f1_sco = f1_score(actual,pred,average='macro')
    return accuracy_sco,confusion_mat,precision_sco,recall_sco,f1_sco

In [52]:
Evaluation_metrics(pred=y_pred,actual=y_test)

(0.6551226551226551,
 array([[208,  10,  14],
        [ 97,  91,  43],
        [ 57,  18, 155]], dtype=int64),
 0.6901411977279185,
 0.6548013871851953,
 0.6405647221333496)